Set the url's for the SensorCity dataportal:

In [4]:
scdp_newjob_url='http://sensorcity-api.kxa.nl/newjob'
scdp_status_url='http://sensorcity-api.kxa.nl/status'
scdp_download_url='http://sensorcity-api.kxa.nl/download'

Get login credentials from the user:

In [2]:
import getpass
scdp_user=getpass.getpass('User name: ')
scdp_pw=getpass.getpass('User password: ')

User name: ········
User password: ········


Prepare json data defining the data to be retrieved from the dataportal:

In [ ]:
# get modules for internet communication and time management
import json
import urllib2
import time

In [34]:
'''
 Define the database query and send it to the dataportal 
'''

# Create a json string containing the query
query_string=json.dumps(
{
    "username": scdp_user,
    "password": scdp_pw,
    "job": {
    "name": "weather-data-export-test",
    "datetime-start": "2016-sep-10 10:00:00",
    "datetime-end": "2016-sep-11 10:01:00",
    "type":"weather",
    "sensors": ["006-WE-1",]
    
  }
  
})

# calculate the length of the json string, this is needed for the webrequest
query_length = len(query_string)

# build the webrequest ... 
req = urllib2.Request(scdp_newjob_url, query_string, {'Content-Type': 'application/json', 'Content-Length': query_length})

# ... and send the webrequest to the data portal
f = urllib2.urlopen(req)

# ... process the returned data
response = f.read()
f.close()

# Get job-id and store it to upload the requested data later
response_dict=json.loads(response)
jobid=response_dict[u'job'][u'job-id']

print('The job id for this query is: {0}'.format(jobid))


The job id for this query is: 1803813


Watch the dataportal for finalization of the job, and then when it is done download the data.

In [43]:
''' Check the status of the job keep polling untill it is finished '''


# Create a json string containing the query

query_string=json.dumps(
{
    "username": scdp_user,
    "password": scdp_pw,
    "job-id": jobid
  }
)

# calculate the length of the json string, this is needed for the webrequest
query_length = len(query_string)

# Do the polling  
waiting=True
while waiting:
        # build the webrequest ... 
        req = urllib2.Request(scdp_status_url, query_string, {'Content-Type': 'application/json', 'Content-Length': query_length})
        # ... and send the webrequest to the data portal
        f = urllib2.urlopen(req)
        # ... process the returned data
        status = f.read()
        f.close()
        status_dict=json.loads(status)
        if status_dict[u'job'][u'status'] == 'finished':
            waiting=False
        else:
            print status_dict[u'job'][u'status']
            time.sleep(3)  
            
            
''' Download the data '''

# calculate the length of the json string, this is needed for the webrequest
query_length = len(query_string)

# build the webrequest ... 
req = urllib2.Request(scdp_download_url, query_string, {'Content-Type': 'application/json', 'Content-Length': query_length})

try:
    # ... and send the webrequest to the data portal
    f = urllib2.urlopen(req)
    
    # ... process the returned data
    # Creating a zip-file this way seens to work only on linux.
    zipfile = open('weather2today.zip','w')
    zipfile.write(f.read())
    zipfile.close()
    f.close()
    
#handle errors
except urllib2.HTTPError, e:
    print "HTTP Error:", e.code, url
except urllib2.URLError, e:
    print "URL Error:", e.reason, url